In [1]:
import pandas as pd

movies = pd.read_excel('dev_set_groundtruth_and_trailers.xls',index_col='filename')


In [2]:
import os
import numpy as np
import xml.etree.cElementTree as et

for filename in os.listdir('XML'):
    tree = et.parse('XML/'+filename)
    root = tree.getroot()
    tmp = root[0]
    
    for i in tmp.attrib:
        movies.loc[filename[:-4],i] = tmp.get(i)

In [3]:
movies.drop(['released','Website','imdbID','poster','tomatoConsensus','trailer','writer','DVD','plot','title','awards'], axis=1, inplace=True)

In [4]:
# NOT REQUIRED

#movies[['BoxOffice']] = movies[['BoxOffice']].replace('[\$,.]','',regex=True)
#movies[['BoxOffice']] = movies[['BoxOffice']].replace(to_replace='M',value='00000',regex=True)
#movies[['BoxOffice']] = movies[['BoxOffice']].replace(to_replace='k',value='00',regex=True)
#movies[['BoxOffice']] = movies[['BoxOffice']].replace('N/A','NaN').astype(np.float)



In [5]:
movies['imdbRating'] = movies['imdbRating'].astype(np.float)
movies['imdbVotes'] = movies['imdbVotes'].replace('[,]','',regex=True).astype(float)
movies['runtime'] = movies['runtime'].replace('[\smin]','',regex=True).astype(float)
movies = movies.replace('N/A','NaN')
movies['rated'] = movies['rated'].replace('NaN','NOT RATED')
movies['year'] = movies['year'].astype(float)
movies['metascore'] = movies['metascore'].astype(float)
movies['tomatoRating'] = movies['tomatoRating'].astype(float)
movies['tomatoUserRating'] = movies['tomatoUserRating'].astype(float)
movies['tomatoMeter'] = movies['tomatoMeter'].astype(float)
movies['tomatoUserMeter'] = movies['tomatoUserMeter'].astype(float)



In [6]:
movies.fillna(0,inplace=True)

In [7]:
metadata = movies[['language','year','genre','country','runtime','rated']]
userrating = movies[['imdbRating','metascore','tomatoRating','tomatoUserRating','tomatoMeter','tomatoUserMeter']]
meta_and_user = movies[['language','year','genre','country','runtime','rated','imdbRating','metascore','tomatoRating',
                       'tomatoUserRating','tomatoMeter','tomatoUserMeter']]

In [8]:
import Encoding as enc

enc.one_hot_encode(metadata,'language',True)
enc.one_hot_encode(metadata,'genre',True)
enc.one_hot_encode(metadata, 'country',True)
enc.one_hot_encode(metadata, 'rated',True)

metadata.drop(['language','country','rated','genre'], axis=1, inplace=True)

C:\Users\alexa\PycharmProjects\Experimental_Design_for_Data_Science\Encoding.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[i] = np.zeros(95)
C:\Users\alexa\PycharmProjects\Experimental_Design_for_Data_Science\Encoding.py:25: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  dataframe.ix[counter, j] += 1
C:\ProgramData\Anaconda3\envs\Experimental_Design_for_Data_Science\lib\site-packages\pandas\core\indexing.py:961: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:

In [9]:
enc.one_hot_encode(meta_and_user,'language',True)
enc.one_hot_encode(meta_and_user, 'country',True)
enc.one_hot_encode(meta_and_user, 'genre',True)
enc.one_hot_encode(meta_and_user, 'rated',True)

meta_and_user.drop(['language','country','rated','genre'], axis=1, inplace=True)

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

knn = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski',
                           metric_params=None,n_jobs=-1)
nc = NearestCentroid(metric='euclidean', shrink_threshold=None)
tree = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1,
                              min_weight_fraction_leaf=0.0, max_features=None, random_state=6, max_leaf_nodes=None,
                              class_weight=None, presort=False)
log = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1,
                         class_weight=None, random_state=6, solver='liblinear', max_iter=100, multi_class='ovr', 
                         verbose=0, warm_start=False, n_jobs=-1)
svm = SVC(C=1.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, probability=False, tol=0.001, 
          cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape=None, random_state=6)
bag = BaggingClassifier(base_estimator=None, n_estimators=10, max_samples=1.0, max_features=1.0, bootstrap=True, 
                        bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=-1, random_state=1, verbose=0)
rf = RandomForestClassifier(n_estimators=10, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, 
                            min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, bootstrap=True, 
                            oob_score=False, n_jobs=-1, random_state=4, verbose=0, warm_start=False, class_weight=None)
ada = AdaBoostClassifier(base_estimator=None, n_estimators=50, learning_rate=1.0, algorithm='SAMME.R', random_state=6)
gb = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100, subsample=1.0, min_samples_split=2,
                                min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, init=None, random_state=6,
                                max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, presort='auto')
gauss = GaussianNB()

In [11]:
from sklearn.model_selection import cross_val_score

algo_pool = [knn,nc,tree,log,svm,bag,rf,ada,gb,gauss]
algo_to_use = []

for i in algo_pool:
    f1 = np.mean(cross_val_score(estimator=i,X=metadata,y=movies.loc[:,'goodforairplane'],
                         scoring='f1',cv=10,n_jobs=-1))
    precision = np.mean(cross_val_score(estimator=i,X=metadata,y=movies.loc[:,'goodforairplane'],
                         scoring='precision',cv=10,n_jobs=-1))
    recall = np.mean(cross_val_score(estimator=i,X=metadata,y=movies.loc[:,'goodforairplane'],
                         scoring='recall',cv=10,n_jobs=-1))
    print(f'Precision: {precision}, Recall: {recall}, f1: {f1}')
    if f1 > 0.5 and precision > 0.5 and recall > 0.5:
        algo_to_use.append(i)
        

Precision: 0.5956349206349205, Recall: 0.6366666666666666, f1: 0.5642607392607393
Precision: 0.5595238095238094, Recall: 0.5366666666666666, f1: 0.5382478632478632
Precision: 0.511904761904762, Recall: 0.6033333333333333, f1: 0.5236541236541237
Precision: 0.5125, Recall: 0.56, f1: 0.5244599844599845
Precision: 0.4950396825396826, Recall: 0.6933333333333334, f1: 0.5606776556776557
Precision: 0.5125, Recall: 0.4666666666666667, f1: 0.46663170163170165
Precision: 0.4966666666666667, Recall: 0.4066666666666666, f1: 0.41601731601731595
Precision: 0.5049206349206349, Recall: 0.5233333333333333, f1: 0.4861754911754912
Precision: 0.48908730158730157, Recall: 0.56, f1: 0.5094194694194694
Precision: 0.45, Recall: 0.13333333333333336, f1: 0.19523809523809527


In [ ]:
import LVW

print(LVW.lvw(meta_and_user,movies.loc[:,'goodforairplane'],80,0.))

In [ ]:
print(LVW.lvw(meta_and_user,movies.loc[:,'goodforairplane'],80,0.))

In [ ]:
print(LVW.lvw(meta_and_user,movies.loc[:,'goodforairplane'],80,0.))

In [ ]:
print(LVW.lvw(movies.drop(['goodforairplane','imdbVotes','tomatoUserReviews','tomatoFresh'],axis=1),movies.loc[:,'goodforairplane'],94,0.))
